In [1]:
import numpy
import xarray
import matplotlib.pyplot as mp
import glob
import pandas
import cartopy
import itertools

import time
#import logging
#logging.captureWarnings(True)
#logging.getLogger('py.warnings').setLevel(logging.ERROR)

import dask_jobqueue
import dask.distributed
import ipywidgets

In [7]:
cluster = dask_jobqueue.PBSCluster(cores=36,
                                   memory='100 GB',
                                   #cores=1,
                                   project='UCLA0022',
                                   queue='regular',
                                   resource_spec='select=1:ncpus=36:mem=100G',
                                   walltime='00:30:00')

/glade/u/home/baird/anaconda3/envs/pangeo/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [9]:
# ask for 10 workers with these specifications
cluster.start_workers(10)

In [10]:
client = dask.distributed.Client(cluster)

distributed.scheduler - INFO - Receive client connection: Client-14237b70-2679-11e9-b21b-bb7b612eebb5
distributed.core - INFO - Starting established connection


In [12]:
!qstat -u baird


chadmin1: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
4212219.chadmin baird    regular  STDIN       71379   1   4    --  12:00 R 04:17
4212222.chadmin baird    regular  STDIN       70408   1   4    --  12:00 R 04:15
4214520.chadmin baird    regular  dask-worke  58022   1  36  100gb 00:30 R 00:00
4214521.chadmin baird    regular  dask-worke  13785   1  36  100gb 00:30 R 00:00
4214522.chadmin baird    regular  dask-worke  72462   1  36  100gb 00:30 R 00:00
4214523.chadmin baird    regular  dask-worke  53946   1  36  100gb 00:30 R 00:00
4214524.chadmin baird    regular  dask-worke   3047   1  36  100gb 00:30 R 00:00
4214525.chadmin baird    regular  dask-worke   1212   1  36  100gb 00:30 R 00:00
4214526.chadmin baird    regular  dask-worke  42202   1  36  100gb 00:30 R 00:00
4214527.chadmin ba

# functions

In [14]:
def slow_increment(x):
    time.sleep(1)
    return x+1
from dask.distributed import progress # progress bar
futures = client.map(slow_increment, range(5000))
progress(futures)

VBox()

# dask arrays

In [15]:
import dask.array as da #numpy interface

x = da.random.random((50000,50000), chunks=(1000,1000))

x = x.persist()

x.nbytes/1e9

20.0

In [16]:
y = (x+x.T) - x.mean(axis=0)
y = y.persist()
y.sum().compute()

1250021808.6324863

# dask dataframes

In [19]:
import dask.dataframe as dd

In [21]:
df = dd.demo.make_timeseries()
df

,id,name,x,y
npartitions=11,,,,
2000-01-31,int64,object,float64,float64
2000-02-29,...,...,...,...
...,...,...,...,...
2000-11-30,...,...,...,...
2000-12-31,...,...,...,...
